In [1]:
import numpy as np
import pandas as pd

In [2]:
import cufflinks as cf
cf.set_config_file(offline=True, world_readable=True, theme='ggplot')
from plotly.offline import init_notebook_mode, iplot
from plotly import graph_objs as go

# Initialize plotly
init_notebook_mode(connected=True)

def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-latest.min.js?noext',
            },
          });
        </script>
        '''))
import IPython

IPython.get_ipython().events.register('pre_run_cell', configure_plotly_browser_state)

In [4]:
df = pd.read_csv("data/AirPassengers.csv")
df['Month'] = pd.to_datetime(df["Month"], infer_datetime_format=True)
df = df.set_index(['Month'])


In [5]:
df.iplot(xTitle="Date", yTitle="Passengers")

## Test if stationar

In [6]:
rollmean = df.rolling(window=12).mean()
rollstd = df.rolling(window=12).std()


In [7]:
from plotly import graph_objs as go

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=pd.to_datetime(rollmean.index.values),
    y=rollmean['#Passengers'],
    name='mean',mode='lines'))

fig.add_trace(go.Scatter(
    x=pd.to_datetime(rollmean.index.values),
    y=rollstd['#Passengers'],
    name='std',mode='lines'))

fig.add_trace(go.Scatter(
    x=pd.to_datetime(df.index.values),
    y=df['#Passengers'],
    name='origin',mode='lines'))

fig.show()

## ADF

In [15]:
from statsmodels.tsa.stattools import adfuller

def adf_test (df):
    dftest = adfuller(df, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used', 'Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput)
    
adf_test(df["#Passengers"])

Test Statistic                   0.815369
p-value                          0.991880
#Lags Used                      13.000000
Number of Observations Used    130.000000
Critical Value (1%)             -3.481682
Critical Value (5%)             -2.884042
Critical Value (10%)            -2.578770
dtype: float64


## KPSS

In [ ]:
from statmodels.tsa.statools import kpss

def kpss_test (df):
    kpsstest = kpss(df)
    kpss_output = pd.Series(kpsstest[0:3], index =['Test Statistic', 'p-value','Lags Used'])
    for key, value in kpsstest[3].items():
        kpssoutput['Critical Value (%s)'%key] = value
        